| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

#### This notebook is licensed under GPL 3.0. Please visit our Github repo for more information: https://github.com/edgi-govdata-archiving/ECHO-COVID19
#### The notebook was collaboratively authored by EDGI following our authorship protocol: https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/
#### For more information about this project, visit https://www.environmentalenforcementwatch.org/

# Use the region.db SQLite database to look up data on regions

This notebook uses the region.db database that is part of the EEW Congressional Report Card generation process.  It gives the user the opportunity to look into the information collected in that database, in ways that may not be a part of our current report cards.

## How to Run
* A "cell" in a Jupyter notebook is a block of code performing a set of actions making available or using specific data.  The notebook works by running one cell after another, as the notebook user selects offered options.
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* **It is important to run cells in order because they depend on each other.**
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---

# **Let's begin!**

Hover over the "[ ]" on the top left corner of the cell below and you should see a "play" button appear. Click on it to run the cell then move to the next one.

These first two cells give us access to some external Python code we will need.

### 1.  Bring in some code that is stored in a Github project.
These two github repositories hold Python code that the notebook uses.
* ECHO_modules holds code that can be used in this and other notebooks--the DataSet class, the make_data_sets() function, etc.
* The ECHO-Cross-Program repository is the one this notebook is contained in.  We clone it to be able to use the utilities.py file contained in it.

In [ ]:
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git
print("Done!")

### 2.  Import the Region class
This class represents a particular region--a state or congressional district. It is constructed with these attributes.

    Attributes
    ----------
    type : str
        One of the supported region types--'State',
        'Congressional District'    
    value : str
        The actual identifier of the region--e.g. the number of
        the congressional district, omit for states    
    state : str
        The two letter state abbreviation    
    programs : str
        The EPA programs


In [2]:
from Region import Region

programs = ['CAA', 'CWA', 'RCRA']

### 3. Begin using functions available through the Region class
    def get_per_1000( self, type, region, year ):
    def get_cwa_per_1000( self, year ):
    def get_recurring_violations( self, program ):
    def get_inflation( self, base_year ):
    def get_events( self, type, program, base_year ):
    def get_non_compliants( self, program ):
    def get_active_facilities( self, program, table='active_facilities' ):

These examples get the violations per 1000 facilities for the total USA and the selected state.

In [3]:
this_state = 'NY'
region = Region( type='State', state=this_state,
                programs=programs )

df = region.get_per_1000( 'violations', 'USA', 2020 )
print( "USA violations" )
print(df)
df = region.get_per_1000( 'violations', 'State', 2020 )
print( "State violations - {}".format(this_state) )
print(df)

USA violations
  Program      Per1000
0     CAA    15.896591
1     CWA  1041.326467
2    RCRA    16.083605
State violations - NY
  Program      Per1000
0     CAA     1.506717
1     CWA  1372.311164
2    RCRA     6.804825


### 4.  Loop through all states and get violations per 1000 facilities for each

In [7]:
import pandas as pd
from ECHO_modules.geographies import states

all_df = pd.DataFrame( columns=['State', 'Program', 'Per1000'] )
for this_state in states:
    region = Region( type='State', state=this_state,
                programs=programs )
    df = region.get_per_1000( 'violations', 'State', 2020 )
    df.insert( 0, 'State', this_state )
    all_df = all_df.append( df, ignore_index=True )
    # print( "State violations - {}".format(this_state) )
    # print(df)
print( all_df )
all_df.to_json( 'state_violations.json', orient='records' )

    State Program      Per1000
0      AL     CAA    12.735327
1      AL     CWA   334.660617
2      AL    RCRA    19.992456
3      AK     CAA   111.888112
4      AK     CWA  9043.868395
..    ...     ...          ...
148    WI     CWA   621.195652
149    WI    RCRA    14.986604
150    WY     CAA     7.228916
151    WY     CWA  1443.783069
152    WY    RCRA    22.260274

[153 rows x 3 columns]


### 4.  Loop through all CDs and get violations per 1000 facilities for each

In [24]:
import pandas as pd

def get_cd( row ):
    region = Region( type='Congressional District', state=row[0], value=row[1],
                programs=programs )
    df = region.get_per_1000( type='violations', region='CD', year=2020 )
    df.insert( 0, 'State', row[0] )
    df.insert( 1, 'CD', row[1] )
    return df
    

region = Region( type='None' )
all_cds = region.get_cds()

all_df = pd.DataFrame( columns=['State', 'CD', 'Program', 'Per1000'] )

all_df = all_df.append( all_cds.apply( get_cd, axis=1 ), ignore_index=True )
all_df.to_json( 'cd_violations.json', orient='records' )

  State   CD Program Per1000  \
0   NaN  NaN     NaN     NaN   

                                                   0  \
0    State  CD Program     Per1000
0    AL   4   ...   

                                                   1  \
0    State  CD Program     Per1000
0    CA  37   ...   

                                                   2  \
0    State  CD Program     Per1000
0    FL  12   ...   

                                                   3  \
0    State  CD Program      Per1000
0    GA   2  ...   

                                                   4  \
0    State  CD Program     Per1000
0    OR   3   ...   

                                                   5  ...  \
0    State  CD Program     Per1000
0    TX   8   ...  ...   

                                                 428  \
0    State  CD Program    Per1000
0    UT   1    ...   

                                                 429  \
0    State  CD Program    Per1000
0    UT   4    ...   

                     

In [22]:
def get_cd( row ):
    region = Region( type='Congressional District', state=row[0], value=row[1],
                programs=programs )
    df = region.get_per_1000( type='violations', region='CD', year=2020 )
    df.insert( 0, 'State', row[0] )
    df.insert( 1, 'CD', row[1] )
    breakpoint()
    return df
    
all_df = all_df.append( all_cds.apply( get_cd, axis=1 ), ignore_index=True )
print( all_df )
# all_df.to_json( 'cd_violations.json', orient='records' )

> <ipython-input-22-4f4a2f0055b9>(8)get_cd()
      6     df.insert( 1, 'CD', row[1] )
      7     breakpoint()
----> 8     return df
      9 
     10 all_df = all_df.append( all_cds.apply( get_cd, axis=1 ), ignore_index=True )

ipdb> df
  State  CD Program     Per1000
0    AL   4     CAA    0.000000
1    AL   4     CWA  468.733007
2    AL   4    RCRA   33.232628
ipdb> all_df
  State   CD Program Per1000  \
0   NaN  NaN     NaN     NaN   
1   NaN  NaN     NaN     NaN   
2   NaN  NaN     NaN     NaN   

                                                   0  \
0                                               None   
1    State  CD Program     Per1000
0    AL   4   ...   
2    State  CD Program     Per1000
0    AL   4   ...   

                                                   1  \
0                                               None   
1    State  CD Program     Per1000
0    CA  37   ...   
2    State  CD Program     Per1000
0    CA  37   ...   

                                           

BdbQuit: 